<font face="B Mitra" size=4>
<div dir=rtl align=center>
<br>
<img src="https://aut.ac.ir/templates/tmpl_modern01/images/logo_fa.png" alt="Amirkabir University Logo" width="100">
<br>
<font size=6>
<b>پروژه دوم داده کاوی (بخش دوم)</b>
<br>
<font size=5> استاد درس: دکتر فاطمه شاکری
<br>
<font size=5> طراحان پروژه: مهدی رجالی، مهدی غیاثی، سینا ارزبین
<br>
<font size=4> پاییز ۱۴۰۳
<hr>
</div>
</font>

<font face="B Nazanin">
<div dir=rtl>
<p align="justify">
<font size=5><font color='#eb4034'>
توجه داشته باشید علاوه بر پیاده سازی خواسته شده، توضیحات لازم در محل تعریف شده را بیان کنید.
</p>
</font>
</div>
</font>

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
بارگذاری کتابخانه های مورد نیاز

In [22]:
import pandas as pd
import numpy as np
import warnings
import category_encoders as ce
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool

warnings.filterwarnings("ignore")

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
بارگذاری مجموعه داده

In [3]:
df = pd.read_csv('GDSC Dataset.csv')

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
جداسازی مجموعه داده‌های Test ،Train و Validation

<div dir=rtl>
<font face="B Nazanin" size=4>
در این بخش می‌خواهیم جداسازی مجموعه داده‌ها را بر حسب ویژگی‌های موجود انجام دهیم.
</font>
</p>


<div dir=rtl>
<h4>
<font face="B Nazanin">
<b>1. جداسازی برحسب نوع بافت در هر دارو</b>
</font>
</h4>
<div dir=rtl>
<font face="B Nazanin" size=4>
تابعی بنویسید که از هر
Drug
تعداد یکسانی
General tissue description
را انتخاب کند و داده‌های مربوط به آن‌ها از مجموعه‌ی آموزشی جدا کرده و در مجموعه‌ی آزمایشی قرار دهد.
</font>
</p>

In [4]:







def tissue_train_test_split(df_train, count, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    df_train = df_train.copy()
    df_test = pd.DataFrame(columns=df_train.columns)

    # گروه‌بندی داده‌ها بر اساس Drug و General tissue description
    grouped = df_train.groupby(['Drug', 'General tissue description'])

    for name, group in grouped:
        # انتخاب تعداد مشخصی از نمونه‌ها به صورت تصادفی، یا همه نمونه‌ها اگر کمتر از count باشند
        n_samples_to_take = min(len(group), count)
        
        if n_samples_to_take > 0:  # بررسی وجود نمونه‌ها
            # انتخاب نمونه‌ها
            test_samples = group.sample(n=n_samples_to_take, random_state=random_state)
            
            # افزودن نمونه‌های انتخاب شده به df_test
            df_test = pd.concat([df_test, test_samples], ignore_index=True)
            
            # حذف نمونه‌های انتخاب شده از df_train
            df_train = df_train.drop(test_samples.index)

    return df_train.reset_index(drop=True), df_test.reset_index(drop=True)

# مثال استفاده
# df_train1, df_test1 = tissue_train_test_split(df, count=3, random_state=42)
# df_train1, df_val1 = tissue_train_test_split(df_train1, count=3, random_state=42)

# نمایش اندازه مجموعه‌ها
# print("Train set size:", len(df_train1))
# print("Validation set size:", len(df_val1))
# print("Test set size:", len(df_test1))

In [5]:
df_train1, df_test1 = tissue_train_test_split(df, count=3, random_state=42)
df_train1, df_val1 = tissue_train_test_split(df_train1, count=3, random_state=42)

print("Train set size:", len(df_train1))
print("Validation set size:", len(df_val1))
print("Test set size:", len(df_test1))

Train set size: 209828
Validation set size: 15991
Test set size: 16216


<div dir=rtl>
<h4>
<font face="B Nazanin">
<b>2. جداسازی برحسب هدف بیولوژیکی در هر مسیر بیولوژیکی</b>
</font>
</h4>
<div dir=rtl>
<font face="B Nazanin" size=4>
تابعی بنویسید که از هر
Target pathway
درصد یکسانی از مقادیر مختلف
Biological target
را انتخاب کند و داده‌های مربوط به آن‌ها از مجموعه‌ی آموزشی جدا کرده و در مجموعه‌ی آزمایشی قرار دهد.
</font>
</p>

In [6]:


def biological_target_train_test_split(df_train, portion, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    df_train = df_train.copy()
    df_test = pd.DataFrame(columns=df_train.columns)

    # گروه‌بندی داده‌ها بر اساس Target pathway و Biological target
    grouped = df_train.groupby(['Target pathway', 'Biological target'])

    for name, group in grouped:
        # محاسبه تعداد نمونه‌ها برای مجموعه آزمایشی
        n_test_samples = int(len(group) * portion)
        
        # انتخاب تصادفی نمونه‌ها
        test_samples = group.sample(n=n_test_samples, random_state=random_state)
        
        # افزودن نمونه‌های انتخاب شده به df_test
        df_test = pd.concat([df_test, test_samples])
        
        # حذف نمونه‌های انتخاب شده از df_train
        df_train = df_train.drop(test_samples.index)

    return df_train.reset_index(drop=True), df_test.reset_index(drop=True)

# مثال استفاده
# df_train: DataFrame شامل داده‌های اولیه
# df_train, df_test = biological_target_train_test_split(df_train, 0.2, random_state=42)

In [7]:
df_train2, df_test2 = biological_target_train_test_split(df, portion=0.2, random_state=42)
df_train2, df_val2 = biological_target_train_test_split(df_train2, portion=0.2, random_state=42)

print("Train set size:", len(df_train2))
print("Validation set size:", len(df_val2))
print("Test set size:", len(df_test2))

Train set size: 164821
Validation set size: 34319
Test set size: 42895


<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
مدیریت مقادیر گم‌شده

<div dir=rtl>
<font face="B Nazanin" size=4>
از تابعی که در بخش اول پروژه پیاده سازی کرده‌اید استفاده کنید.
</font>
</p>

In [8]:
train_imputation_data = {}

In [9]:
def get_mode(group, column, calculate_mode, default_val=np.nan):
    if calculate_mode:
        mode_val = group.mode()
        train_imputation_data[f'{group.name}_{column}'] = mode_val[0] if not mode_val.empty else default_val
    
    return train_imputation_data.get(f'{group.name}_{column}', default_val)

In [10]:
def impute_missing_values(df: pd.DataFrame, train_set: bool):
    df = df.copy()
    
    # Imputation of 'Cancer Type' column
    df['Cancer Type'] = df.groupby(by=['Drug', 'Specific tissue description'], dropna=False)['Cancer Type'].transform(
                            lambda group: group.fillna(get_mode(group, column='Cancer Type', calculate_mode=train_set))
                        )

    df['Cancer Type'] = df.groupby(by=['Drug', 'General tissue description'], dropna=False)['Cancer Type'].transform(
                            lambda group: group.fillna(get_mode(group, column='Cancer Type', calculate_mode=train_set))
                        )
    

    # Imputation of 'Specific tissue description' and 'General tissue description' column
    columns_to_fill = ['General tissue description', 'Specific tissue description']
    for col in columns_to_fill:
        df[col] = df.groupby(by=['Drug', 'Cancer Type'], dropna=False)[col].transform(
                                lambda group: group.fillna(get_mode(group, column=col, calculate_mode=train_set))
                            )
        
    # Imputation of 'Biological target' and 'Target pathway' column
    columns_to_fill = ['Biological target', 'Target pathway']
    for col in columns_to_fill:
        df[col] = df.groupby(by=['Drug'], dropna=False)[col].transform(
                                lambda group: group.fillna(get_mode(group, column=col, calculate_mode=train_set))
                            )
        
    # Imputation of ['Microsatellite instability Status', 'Screen Medium', 'Growth Properties', 'CNA', 'Gene Expression', 'Methylation'] column
    columns_to_fill = ['Microsatellite instability Status', 'Screen Medium', 'Growth Properties', 'CNA', 'Gene Expression', 'Methylation']
    for col in columns_to_fill:
        df[col] = df.groupby(by=['Drug', 'General tissue description'], dropna=False)[col].transform(
                                lambda group: group.fillna(get_mode(group, column=col, calculate_mode=train_set))
                            )
        
    
    df.fillna('Unknown', inplace=True)

    return df

In [11]:
df_train1 = impute_missing_values(df=df_train1, train_set=True)
df_val1 = impute_missing_values(df=df_val1, train_set=False)
df_test1 = impute_missing_values(df=df_test1, train_set=False)

In [12]:
df_train2 = impute_missing_values(df=df_train2, train_set=True)
df_val2 = impute_missing_values(df=df_val2, train_set=False)
df_test2 = impute_missing_values(df=df_test2, train_set=False)

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
آموزش مدل:

<div dir=rtl>
<h4>
<font face="B Nazanin">
<b>تفکیک ویژگی‌ها و برچسب‌ها</b>
</font>
</h4>

In [38]:
features = df_train1.iloc[:, :-1] 
target = df_train1.iloc[:, -1] 

X_train1 = features
Y_train1 = target

X_test1 = df_test1.iloc[:, :-1]
Y_test1 = df_test1.iloc[:, -1]

X_valid1 = df_val1.iloc[:, :-1]
Y_valid1 = df_val1.iloc[:, -1]

In [37]:


# Also for validation and test sets

In [45]:
X_train2 = df_train2.iloc[:, :-1]
Y_train2 = target = df_train2.iloc[:, -1] 

X_test2 = df_test2.iloc[:, :-1]
Y_test2 = df_test2.iloc[:, -1]

X_valid2 = df_val2.iloc[:, :-1]
Y_valid2 = df_val2.iloc[:, -1]
# Also for validation and test sets

<div dir=rtl>
<h4>
<font face="B Nazanin">
<b>1. مدل CatBoost با جداسازی اول</b>
</font>
</h4>

In [40]:
categorical_features = [
    'Cell line identifier', 'Drug', 'General tissue description',
    'Specific tissue description', 'Cancer Type', 
    'Microsatellite instability Status', 'Screen Medium', 
    'Growth Properties', 'CNA', 'Gene Expression', 
    'Methylation', 'Biological target', 'Target pathway'
]
train_pool= Pool(data=X_train1, label=Y_train1, cat_features=categorical_features)
val_pool = Pool(data=X_valid1, label=Y_valid1,cat_features=categorical_features)

In [43]:
model1 = CatBoostRegressor(
    
    iterations=500,           
    learning_rate=0.05,       
    depth=6,                  
    loss_function='RMSE',     
    random_seed=42,           
    verbose=50,
    task_type="GPU", # Uncomment for faster training if a dedicated GPU is available
    devices='0',
    cat_features=categorical_features
   
)

model1.fit(
    train_pool,               
    eval_set=val_pool,        
    early_stopping_rounds=50
)

0:	learn: 2.6775368	test: 2.6673836	best: 2.6673836 (0)	total: 131ms	remaining: 1m 5s
50:	learn: 1.4372061	test: 1.4225192	best: 1.4225192 (50)	total: 6.68s	remaining: 58.8s
100:	learn: 1.3256074	test: 1.2988118	best: 1.2988118 (100)	total: 13.1s	remaining: 51.7s
150:	learn: 1.2845064	test: 1.2563081	best: 1.2563081 (150)	total: 19.6s	remaining: 45.4s
200:	learn: 1.2619673	test: 1.2331108	best: 1.2331108 (200)	total: 26.2s	remaining: 39s
250:	learn: 1.2451914	test: 1.2145906	best: 1.2145906 (250)	total: 33.3s	remaining: 33.1s
300:	learn: 1.2334617	test: 1.2035207	best: 1.2035207 (300)	total: 39.8s	remaining: 26.3s
350:	learn: 1.2246255	test: 1.1954764	best: 1.1954764 (350)	total: 46.6s	remaining: 19.8s
400:	learn: 1.2151627	test: 1.1858272	best: 1.1858272 (400)	total: 53.1s	remaining: 13.1s
450:	learn: 1.2088406	test: 1.1800379	best: 1.1800379 (450)	total: 59.8s	remaining: 6.49s
499:	learn: 1.2036960	test: 1.1751825	best: 1.1751825 (499)	total: 1m 6s	remaining: 0us
bestTest = 1.1751825

In [44]:
from sklearn.metrics import mean_squared_error
train_predictions1 = model1.predict(train_pool)
val_predictions1 = model1.predict(val_pool)
test_predictions1 = model1.predict(X_test1)


<div dir=rtl>
<h4>
<font face="B Nazanin">
<b>2. مدل CatBoost با جداسازی دوم</b>
</font>
</h4>

In [47]:
categorical_features = [
    'Cell line identifier', 'Drug', 'General tissue description',
    'Specific tissue description', 'Cancer Type', 
    'Microsatellite instability Status', 'Screen Medium', 
    'Growth Properties', 'CNA', 'Gene Expression', 
    'Methylation', 'Biological target', 'Target pathway'
]
train_pool= Pool(data=X_train2, label=Y_train2, cat_features=categorical_features)
val_pool = Pool(data=X_valid2, label=Y_valid2,cat_features=categorical_features)

In [48]:
model2 = CatBoostRegressor(
    iterations=500,           
    learning_rate=0.05,       
    depth=6,                  
    loss_function='RMSE',     
    random_seed=42,           
    verbose=50,
    task_type="GPU", # Uncomment for faster training if a dedicated GPU is available
    devices='0',
    cat_features=categorical_features
   
)

model2.fit(
    train_pool,               
    eval_set=val_pool,        
    early_stopping_rounds=50
)

0:	learn: 2.6408339	test: 2.7423284	best: 2.7423284 (0)	total: 103ms	remaining: 51.2s
50:	learn: 1.4264389	test: 1.4243224	best: 1.4243224 (50)	total: 5.16s	remaining: 45.5s
100:	learn: 1.3267956	test: 1.3030025	best: 1.3030025 (100)	total: 10.2s	remaining: 40.4s
150:	learn: 1.2942269	test: 1.2613369	best: 1.2613369 (150)	total: 15.4s	remaining: 35.5s
200:	learn: 1.2673949	test: 1.2271068	best: 1.2271068 (200)	total: 20.6s	remaining: 30.6s
250:	learn: 1.2497036	test: 1.2057585	best: 1.2057585 (250)	total: 25.8s	remaining: 25.6s
300:	learn: 1.2392940	test: 1.1938742	best: 1.1938742 (300)	total: 31.2s	remaining: 20.6s
350:	learn: 1.2294609	test: 1.1826493	best: 1.1826493 (350)	total: 36.3s	remaining: 15.4s
400:	learn: 1.2205597	test: 1.1724761	best: 1.1724761 (400)	total: 41.5s	remaining: 10.3s
450:	learn: 1.2132844	test: 1.1653590	best: 1.1653590 (450)	total: 47.1s	remaining: 5.12s
499:	learn: 1.2073077	test: 1.1592885	best: 1.1592885 (499)	total: 52.1s	remaining: 0us
bestTest = 1.15928

In [49]:
from sklearn.metrics import mean_squared_error
train_predictions2 = model2.predict(train_pool)
val_predictions2 = model2.predict(val_pool)
test_predictions2 = model2.predict(X_test2)

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
ارزیابی مدل‌ها:

<div dir=rtl>
<font face="B Nazanin" size=4>
<code>R2 Score</code> را برای هر مدل و بر روی هر کدام از مجموع داده‌ها محاسبه کنید.
</font>
</p>

In [50]:
def evaluate_model(y_true, y_pred, dataset_name):
    r2 = r2_score(y_true, y_pred)
    print(f"{dataset_name} - R2 Score: {r2:.4f}")

evaluate_model(Y_train1, train_predictions1, "CatBoost - Train")
evaluate_model(Y_valid1, val_predictions1, "CatBoost - Validation")
evaluate_model(Y_test1, test_predictions1, "CatBoost - Test")


CatBoost - Train - R2 Score: 0.8448
CatBoost - Validation - R2 Score: 0.8179
CatBoost - Test - R2 Score: 0.8128


In [51]:
def evaluate_model(y_true, y_pred, dataset_name):
    r2 = r2_score(y_true, y_pred)
    print(f"{dataset_name} - R2 Score: {r2:.4f}")

evaluate_model(Y_train2, train_predictions2, "CatBoost - Train")
evaluate_model(Y_valid2, val_predictions2, "CatBoost - Validation")
evaluate_model(Y_test2, test_predictions2, "CatBoost - Test")



CatBoost - Train - R2 Score: 0.8368
CatBoost - Validation - R2 Score: 0.8326
CatBoost - Test - R2 Score: 0.8319


<div dir=rtl>
<font face="B Nazanin" size=4>
نتایج به دست آمده را بر اساس Feature Importance‌های بدست‌آمده در بخش قبل توجیه کنید.
به نظر شما دلیل این کاهش بسیار شدید در کارایی مدل با وجود ویژگی‌های پراهمیت‌تری که هنوز موجود هستند چیست؟

به نظرتان برای پیش‌بینی دارو‌های جدید و یا دارو‌هایی که روی نمونه‌های سلولی ناآشنا (Cell Line‌های جدید) در آزمایشگاه بررسی شده‌اند، 
می‌توان مدلی با کارایی خوب با ویژگی‌های موجود ارائه داد؟

همچنین به نظر شما بین دو مدل CatBoost و Random Forest آموزش داده‌شده در بخش قبل، کدام یک در رویارویی با نمونه‌هایی با دارو‌های ناشناخته یا هدف‌های زیستی جدید مناسب خواهد بود؟ چرا؟
</font>
</p>


<font face="B Nazanin">
<div dir=rtl>
<p align="justify">
<font size=4>
<b> <font color='#eb4034'> توضیحات پاسخ: </font></b>  لطفا اینجا بنویسید
</p>
</font>
</div>
</font>
<hr>

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#FF8F00">
سوال امتیازی

<div dir=rtl>
<font face="B Nazanin" size=4>
یکی از مشکلات اساسی که برای شیوهٔ رمز‌گذاری One Hot مطرح کردیم، افزایش تعداد ویژگی‌ها و پربعد کردن داده‌ها بود.

ما در گذشته یک شیوه برای کاهش بعد داده‌ها مطالعه کردیم که الگوریتم PCA بود.
سوالی که ممکن است پیش بیاید این است که چرا با روش One Hot رمزگذاری نکنیم و سپس با PCA با افزایش بعد ناشی از این رمز‌گذاری مبارزه کنیم؟

این کار، یعنی استفاده از PCA پس از رمز‌گذاری داده‌ها به شیوهٔ One Hot، به چند دلیل صحیح نیست.
در مورد آن مطالعه کنید و با استدلال هر یک از دلایل خود را توضیح دهید.

دقت کنید که دلایل مد نظر مرتبط با مطالبی هستند که شما مطالعه کرده‌اید، و دلایلی مانند پایداری محاسباتی و ... مد نظر نیستند.
</font>
</p>
